In [ ]:
import numpy as np
from sklearn import preprocessing
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [ ]:
df = pd.read_csv("../../data/frc_data/2024/test_2024.csv")

scores = {}
count = 0
for x in df['team']:
    
    scores[x] = (.182 * df['norm_a_c_s_p'][count]) + (.221 * df['norm_a_g_p'][count]) + (.174 * df['norm_c_p'][count]) + (.222 * df['norm_co_p'][count]) + (.202 * df['norm_eg_p'][count])
    count += 1

{k: v for k, v in sorted(scores.items(), key=lambda item: item[1], reverse=True)}

In [ ]:
red_team, blue_team, won = [], [], []

jqd = pd.read_csv("../../data/frc_data/2024/all_qual_data.csv")

iterate = 0

for l in range(len(jqd['red_1'])):

    red_team.append(scores[jqd["red_1"][iterate]] + scores[jqd["red_2"][iterate]] + scores[jqd["red_3"][iterate]])
    blue_team.append(scores[jqd["blue_1"][iterate]] + scores[jqd["blue_2"][iterate]] + scores[jqd["blue_3"][iterate]])
    if(jqd['win'][iterate] == "blue"):
        won.append(1)
    else:
        won.append(0)
    
    iterate += 1

feature_data = pd.DataFrame({
    'red_team': red_team,
    'blue_team': blue_team
})

# Create a Series for the target data
target_data = pd.Series(won, name='target')

# Combine into one DataFrame if needed
combined_data = feature_data.assign(target=target_data)

# Now combined_data is a DataFrame ready for use in machine learning
print(combined_data)

In [ ]:
X = feature_data
y = combined_data['target']

combined_data['target'].value_counts()
X_train, X_test, y_train, y_test = train_test_split(X, y , test_size=.25, random_state=0)

ss_train = StandardScaler()
X_train = ss_train.fit_transform(X_train)

ss_test = StandardScaler()
X_test = ss_test.fit_transform(X_test)

model = LogisticRegression()

model.fit(X_train, y_train)

prob_predictions = model.predict_proba(X_test)
class_predictions = model.predict(X_test)

# Loop through the predictions and print required information
for i, (prob, prediction, actual) in enumerate(zip(prob_predictions, class_predictions, y_test), start=1):
    # Probability of the positive class (assuming it is the second one)
    prob_of_one = prob[1] * 100
    print(f'Match #{i}: Predicted Class: {prediction}, Actual Class: {actual}, Probability of being 1: {prob_of_one:.2f}%')

In [ ]:
cm = confusion_matrix(y_test, class_predictions)

TN, FP, FN, TP = confusion_matrix(y_test, class_predictions).ravel()

print('True Positive(TP)  = ', TP)
print('False Positive(FP) = ', FP)
print('True Negative(TN)  = ', TN)
print('False Negative(FN) = ', FN)

accuracy =  (TP + TN) / (TP + FP + TN + FN)

print('Accuracy of the binary classifier = {:0.3f}'.format(accuracy))